In [1]:
!pip install torch_geometric
# !pip install torch_sparse

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import json
import scipy.sparse as sp

In [3]:
adjacency_matrix = sp.load_npz('adj.npz')

# Load your feature matrix
feature_matrix = np.load('features.npy')
feature_matrix = torch.tensor(feature_matrix, dtype=torch.float)

# Load your labels
labels = np.load('labels.npy')
labels = torch.tensor(labels, dtype=torch.long)

# Load your data splits
with open('splits.json', 'r') as f:
    data_splits = json.load(f)

train_idx = data_splits['idx_train']
test_idx = data_splits['idx_test']

# Extract edge indices from the adjacency matrix
edge_index = from_scipy_sparse_matrix(adjacency_matrix)

In [4]:
# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, data.edge_index[0])
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [5]:
# Initialize model and optimizer
model = GCN(input_dim=feature_matrix.size(1), hidden_dim=64, output_dim=labels.max().item() + 1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
def train(model, data, optimizer, train_mask, epochs):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

In [98]:
train(model, data, optimizer, train_mask, epochs=100)

ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.